# 1. Exploração Inicial:

## 1.1 Dados: `ferramentas que não acompanham Contentor`

In [1]:
# import libraries
import pandas as pd
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

#remove warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
# ler csv
df_toolings = pd.read_csv(r"/content/listagem_ferramentas_movimentacao_sem_container.csv")


In [3]:
total_registos = len(df_toolings)
nomes_unicos = df_toolings['Tool'].nunique()

print(f"Total de registos de ferramentas: {total_registos}")
print(f"Número de nomes únicos: {nomes_unicos}")

Total de registos de ferramentas: 341380
Número de nomes únicos: 4518


In [4]:
df_toolings.Tool.value_counts().sort_values(ascending=True)

,count
Tool,
Armário 12 Cacifos,1
Cinta Plana 2 Ton 4 Metros - PERDIDO CT 28,1
"Jogo de Micrometros de Interior (5 Pcs) 1000-3000mm 0,01mm",1
"Aparafusadora de Impacto a Bateria 3/4"" - Perdido Contentor 12",1
Chave Inserto Boca 41,1
...,...
"Puxa 0,75 Ton (Diferencial Manual Alavanca / Camelão)",5330
Arnês de Segurança,8615
Antiqueda com chicote em V com amortecedor/absorvedor,8625


- Removemos colunas com ferramentas com nomes incorretos:

In [5]:
#remove columns with instruction to Don't use.
ids_to_drop = [5368, 27407, 29248, 5379, 5367, 5380, 7861, 19106]

df_toolings_cleaned = df_toolings[~df_toolings['ToolID'].isin(ids_to_drop)]


In [6]:
df_toolings_cleaned

,MOVEMENTID,ToolID,Tool,TotalVolume,SubFamilyID
0,13,2238,Retificadora diam. 125 mm,1.0,863
1,13,2255,Retificadora diam. 230 mm,1.0,866
2,14,2272,Aparelho de soldar,1.0,527
3,14,2251,Aparelho de Soldar / Arc-air,1.0,529
4,14,2271,Aparelho de soldar,1.0,527
...,...,...,...,...,...
341375,84985,22886,Riscador Tipo Caneta 150 mm,3.0,865
341376,84985,23500,Alicate Freios Ext. Pontas Direitas 125 mm,1.0,1247
341377,84985,24200,Lima paralela Murça com cabo,1.0,728
341378,84985,38467,"Extensor 125mm 1/2""",1.0,1282


In [7]:
import pandas as pd

total_registos = len(df_toolings_cleaned)
nomes_unicos = df_toolings_cleaned['Tool'].nunique()

print(f"Total de registos de ferramentas: {total_registos}")
print(f"Número de nomes únicos: {nomes_unicos}")

Total de registos de ferramentas: 341296
Número de nomes únicos: 4513


In [8]:
print(f"Total de nulos:{df_toolings_cleaned.isnull().sum()}")

Total de nulos:MOVEMENTID     0
ToolID         0
Tool           0
TotalVolume    0
SubFamilyID    0
dtype: int64


## 1.2. Pre-Processamento:

- Primeira limpeza:

In [9]:
import re

def pre_clean_tool_name(name):
    if not isinstance(name, str):
        return "" # Retorna uma string vazia para valores não-texto

    name = name.lower()

    # 1. Remover tudo o que vem depois de um padrão de "lixo"
    # Ex: "retificadora - perdido contentor" -> "retificadora "
    name = re.split(r'[-_]\s*(perdido|danificado|avariado|sanga|ct\d+)', name, 1)[0]

    # 2. Remover datas
    name = re.sub(r'\s*\d{1,2}/\d{1,2}/\d{4}\s*', '', name)

    # 3. Remover códigos entre parêntesis
    name = re.sub(r'\s*\(.*?\)\s*', '', name)

    # 4. Padronizar abreviações e espaçamento
    name = re.sub(r'diam\.', 'diametro', name)
    name = re.sub(r'c/', 'com', name)
    name = re.sub(r's/', 'sem', name)

    # 5. Remover pontuação e espaços extra
    name = re.sub(r'[^\w\s]', ' ', name) # Remove tudo exceto letras, números e espaços
    name = re.sub(r'\s+', ' ', name).strip() # Remove espaços múltiplos

    return name

# Aplique esta pré-limpeza
df_toolings_cleaned['Tool_Pre_Cleaned'] = df_toolings_cleaned['Tool'].apply(pre_clean_tool_name)

# Veja a magia acontecer:
novos_nomes_unicos = df_toolings_cleaned['Tool_Pre_Cleaned'].nunique()
print(f"Número de nomes únicos ANTES da pré-limpeza: {nomes_unicos}")
print(f"Número de nomes únicos DEPOIS da pré-limpeza: {novos_nomes_unicos}")

Número de nomes únicos ANTES da pré-limpeza: 4513
Número de nomes únicos DEPOIS da pré-limpeza: 3486


#### 1.1.1. Lista de nomes únicos:

In [10]:
lista_nomes_sujos = df_toolings_cleaned['Tool'].unique()
lista_nomes_prelimpos = df_toolings_cleaned['Tool_Pre_Cleaned'].unique()

In [11]:
lista_nomes_prelimpos

array(['retificadora diametro 125 mm', 'retificadora diametro 230 mm',
       'aparelho de soldar', ..., 'chave gancho 14 24',
       'contentor de ferramenta nº77', 'caçonete tarracha hss m10'],
      dtype=object)

#### 1.1.1. Aplicando Mapeamento para Correção (Agrupamento por Similaridade (Clustering))

Temos muitas ferramentas com nomes contendo informações irrelevantes, vamos criar um mapeamento para limpar estes dados:

In [12]:
!pip install fuzzywuzzy

In [13]:
# --- ETAPA 1: CRIAR O FICHEIRO DE ENTRADA ---

# Contar a frequência dos nomes pré-limpos
pre_cleaned_counts = df_toolings_cleaned['Tool_Pre_Cleaned'].value_counts()

# O nome do ficheiro que o próximo script vai usar como entrada
NOME_FICHEIRO_INPUT = 'nomes_para_mapear.csv'

# Exportar para um ficheiro CSV. Este ficheiro terá duas colunas: o nome e a contagem.
pre_cleaned_counts.to_csv(NOME_FICHEIRO_INPUT)

print(f"\nFicheiro de entrada '{NOME_FICHEIRO_INPUT}' criado com sucesso!")
print("Agora, vamos usar este ficheiro na Etapa 2.")


Ficheiro de entrada 'nomes_para_mapear.csv' criado com sucesso!
Agora, vamos usar este ficheiro na Etapa 2.


In [14]:
from fuzzywuzzy import fuzz

# --- CONFIGURAÇÕES ---
# O nome do ficheiro da Etapa 1
INPUT_CSV = 'nomes_para_mapear.csv'
# O nome do ficheiro de saída que este script vai gerar
OUTPUT_CSV = 'mapa_ferramentas_sugerido.csv'
# Limiar de similaridade. 85 é um bom valor inicial.
SIMILARITY_THRESHOLD = 85

# --- LÓGICA DO SCRIPT ---

print("Iniciando o script de agrupamento...")

# 1. Carregar os nomes únicos pré-limpos
try:
    df_counts = pd.read_csv(INPUT_CSV)
    df_counts.columns = ['Nome_Pre_Limpado', 'Contagem']
except FileNotFoundError:
    print(f"ERRO: O ficheiro '{INPUT_CSV}' não foi encontrado. Verifique se ele existe na pasta.")
    exit()

df_counts = df_counts.sort_values(by='Contagem', ascending=False)
tools_to_process = df_counts['Nome_Pre_Limpado'].tolist()

print(f"Agrupando {len(tools_to_process)} nomes únicos...")

mapping_list = []

while tools_to_process:
    anchor_name = tools_to_process.pop(0)

    mapping_list.append({
        'Nome_Sujo': anchor_name,
        'Nome_Canonico_Sugerido': anchor_name,
        'Similaridade': 100
    })

    matches = [
        tool for tool in tools_to_process
        if fuzz.token_set_ratio(anchor_name, tool) > SIMILARITY_THRESHOLD
    ]

    for match in matches:
        similarity_score = fuzz.token_set_ratio(anchor_name, match)
        mapping_list.append({
            'Nome_Sujo': match,
            'Nome_Canonico_Sugerido': anchor_name,
            'Similaridade': similarity_score
        })
        tools_to_process.remove(match)

    if len(tools_to_process) % 200 == 0 and len(tools_to_process) > 0:
        print(f"Faltam {len(tools_to_process)} nomes para processar...")

df_map_sugestao = pd.DataFrame(mapping_list)
df_map_sugestao.to_csv(OUTPUT_CSV, index=False)

print(f"\nCONCLUÍDO! O mapa de sugestões foi salvo em '{OUTPUT_CSV}'.")
print("O seu próximo passo é abrir este ficheiro no Excel/Sheets para rever e corrigir.")

Iniciando o script de agrupamento...
Agrupando 3486 nomes únicos...
Faltam 2800 nomes para processar...
Faltam 2200 nomes para processar...
Faltam 800 nomes para processar...
Faltam 600 nomes para processar...
Faltam 200 nomes para processar...

CONCLUÍDO! O mapa de sugestões foi salvo em 'mapa_ferramentas_sugerido.csv'.
O seu próximo passo é abrir este ficheiro no Excel/Sheets para rever e corrigir.


**O ficheiro anterior tem de ver avaliado manualmente.**

In [15]:

# Carregar o mapa final nome canonico/id_final
df_mapa_final = pd.read_csv('/content/mapa_final.csv')

# Manter apenas as colunas necessárias do mapa para a junção
df_mapa_final = df_mapa_final[['Nome_Sujo', 'Nome_Canonico_Final', 'Tool_ID_Final']]
# Renomear 'Nome_Sujo' para corresponder à coluna em df_movimentos
df_mapa_final = df_mapa_final.rename(columns={'Nome_Sujo': 'Tool_Pre_Cleaned'})


# Juntar (merge) a tabela de movimentos com o mapa final
# Isto é como um PROCV em massa
df_movimentos_final = pd.merge(
    df_toolings_cleaned,
    df_mapa_final,
    on='Tool_Pre_Cleaned',
    how='left' # 'left' para garantir que nenhum movimento seja perdido
)



In [16]:
df_movimentos_final.columns

Index(['MOVEMENTID', 'ToolID', 'Tool', 'TotalVolume', 'SubFamilyID',
       'Tool_Pre_Cleaned', 'Nome_Canonico_Final', 'Tool_ID_Final'],
      dtype='object')

In [17]:
# Verificar se algum nome não foi mapeado (deve ser raro)
nao_mapeados = df_movimentos_final[df_movimentos_final['Tool_ID_Final'].isnull()]
if not nao_mapeados.empty:
    print("AVISO: Alguns nomes não foram encontrados no mapa final:")
    print(nao_mapeados['Tool_Pre_Cleaned'].value_counts())

# Selecionar as colunas finais e remover as que não precisa mais
df_final_para_modelo = df_movimentos_final[['MOVEMENTID', 'Tool_ID_Final', 'Nome_Canonico_Final', 'SubFamilyID']]


# Salvar o resultado!
df_final_para_modelo.to_csv('movimentos_limpos_para_modelo.csv', index=False)

print("Processo concluído! O ficheiro 'movimentos_limpos_para_modelo.csv' está pronto para ser usado no sistema de recomendação.")

Processo concluído! O ficheiro 'movimentos_limpos_para_modelo.csv' está pronto para ser usado no sistema de recomendação.


In [18]:
df_final_para_modelo

,MOVEMENTID,Tool_ID_Final,Nome_Canonico_Final,SubFamilyID
0,13,900,retificadora diametro 125 mm,863
1,13,900,retificadora diametro 125 mm,866
2,14,57,aparelho de soldar,527
3,14,57,aparelho de soldar,529
4,14,57,aparelho de soldar,527
...,...,...,...,...
341291,84985,903,riscador tipo caneta 150 mm,865
341292,84985,31,alicate freios ext pontas direitas 125 mm,1247
341293,84985,626,lima paralela murça com cabo,728
341294,84985,479,extensor 125mm 1 2,1282


### 1.1.2. 2ª estratégia: Top K ferramentas + FPGrowth

##### - 1º teste: `k=200` e `min_support=0.01`

```python
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

# df_encoded: seu dataframe one-hot (31.824 x 4.609)

# ----------------------------
# 1) Selecionar os Top-K itens mais frequentes
# ----------------------------
K = 200   # você pode ajustar (100, 500, 1000...)
item_counts = df_encoded.sum(axis=0).sort_values(ascending=False)
top_items = item_counts.head(K).index

# dataframe reduzido
df_top = df_encoded[top_items]

print(f"Rodando FP-Growth em {df_top.shape[1]} itens (Top-{K})")

# ----------------------------
# 2) Rodar FP-Growth apenas nesses itens
# ----------------------------
min_support = 0.01  # ajuste conforme necessário
fi = fpgrowth(df_top, min_support=min_support, use_colnames=True, max_len=3)  
# max_len=3 -> encontra até trios (ajuste para 2 se quiser apenas pares)

print("Itemsets frequentes encontrados:", fi.shape[0])
print(fi.head())

# ----------------------------
# 3) Gerar regras de associação
# ----------------------------
rules = association_rules(fi, metric="lift", min_threshold=1.0)

# ordenar pelas regras mais interessantes
rules = rules.sort_values(["lift", "confidence"], ascending=False).reset_index(drop=True)

print("Regras geradas:", rules.shape[0])
print(rules.head(10))
```


### 1.1.3. **2ª EStratégia: Apenas FP-Growth + Regras de Associação**

In [19]:
# Principais estatísticas:
print(f"Total of registers: {df_final_para_modelo.shape[0]} rows")
print(f"Total of distinct toolings: {df_final_para_modelo.Nome_Canonico_Final.nunique()} toolings")
print (f"Total of duplicates registers: {df_final_para_modelo.duplicated().sum()}\n")
print("="* 80)
print(f"\nTotal of missing values per column: {df_final_para_modelo.isnull().sum()}")

Total of registers: 341296 rows
Total of distinct toolings: 1037 toolings
Total of duplicates registers: 166582


Total of missing values per column: MOVEMENTID             0
Tool_ID_Final          0
Nome_Canonico_Final    0
SubFamilyID            0
dtype: int64


### 1.1.4. 3ª Estratégia: FpGrowth + Association Rules + Business Rules + Graph Community

#### Usando o Nome Canonico:

In [20]:
# group by movements list

transactions_without_container = df_final_para_modelo.groupby('MOVEMENTID')['Nome_Canonico_Final'].apply(list)

In [21]:
transactions_without_container

,Nome_Canonico_Final
MOVEMENTID,
13,"[retificadora diametro 125 mm, retificadora di..."
14,"[aparelho de soldar, aparelho de soldar, apare..."
15,[berbequim de base magnética]
16,[aparelho de soldar]
17,"[aparelho de soldar, retificadora diametro 125..."
...,...
84929,"[chave de bocas 12 13, chave de luneta 24 27, ..."
84930,"[tarraxa elétrica 1 8 2, retificadora diametro..."
84980,"[chave de luneta 24 27, chave de luneta 18 19,..."


In [22]:
transactions_without_container.tolist()

[['retificadora diametro 125 mm', 'retificadora diametro 125 mm'],
 ['aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar'],
 ['berbequim de base magnética'],
 ['aparelho de soldar'],
 ['aparelho de soldar',
  'retificadora diametro 125 mm',
  'retificadora diametro 125 mm',
  'retificadora diametro 125 mm'],
 ['aparelho de soldar'],
 ['retificadora diametro 125 mm'],
 ['retificadora diametro 125 mm'],
 ['rebolo'],
 ['aparelho de soldar'],
 ['berbequim de base magnética'],
 ['aparelho de soldar'],
 ['aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar',
  'aparelho de soldar'],
 ['retificadora diametro 125 mm'],
 ['retificadora diametro 125 mm',
  'retificadora diametro 125 mm',
  'retificadora diametro 125 mm',
  'retificadora diametro 125 mm',
  'retificadora diametro 125 mm',
  'aparelho de soldar',
  'retificadora diametro 125 mm',
  'aparelho de soldar',
  

In [23]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions_without_container).transform(transactions_without_container)
df_transaction_without_container = pd.DataFrame(te_ary, columns=te.columns_)



In [ ]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import pandas as pd

# ------------------------------
# 1. Frequent itemsets
# ------------------------------
frequent_itemsets = fpgrowth(df_transaction_without_container, min_support=0.01, use_colnames=True)
print(frequent_itemsets.head())

# ------------------------------
# 2. Association rules
# ------------------------------
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)

# ordenar pelas regras mais interessantes
rules = rules.sort_values(["lift", "confidence"], ascending=False).reset_index(drop=True)

print("Regras geradas:", rules.shape[0])
print(rules.head(10))

# ------------------------------
# 3. Classificação por lift/confiança
# ------------------------------
strong_rules = rules[(rules['lift'] > 3) & (rules['confidence'] > 0.7)].copy()
contextual_rules = rules[(rules['lift'] > 1) & (rules['lift'] <= 3)].copy()
incompatibles = rules[rules['lift'] < 1].copy()

# ------------------------------
# 4. Criar grafo e detectar comunidades
# ------------------------------
G = nx.Graph()
for _, row in strong_rules.iterrows():
    # antecedents e consequents são frozenset, converte para string
    antecedents = [str(a) for a in row['antecedents']]
    consequents = [str(c) for c in row['consequents']]
    for ant in antecedents:
        for cons in consequents:
            G.add_edge(ant, cons, weight=row['lift'])

communities = list(greedy_modularity_communities(G))

# ------------------------------
# 3. Criar DataFrame de itens com seus grupos
# ------------------------------
item_para_grupo = {}
for i, comm in enumerate(communities):
    for item in comm:
        item_para_grupo[item] = i

df_itens_grupo = pd.DataFrame({
    'item': list(item_para_grupo.keys()),
    'grupo': list(item_para_grupo.values())
})



In [ ]:

# ------------------------------
# 5. Classificação final das regras
# ------------------------------
def classificar_recomendacao(rule):
    if rule['lift'] > 3 and rule['confidence'] > 0.6:
        return "Compatibilidade forte"
    elif rule['lift'] > 1:
        return "Contextual"
    else:
        return "Incompatibilidade"

rules['classe'] = rules.apply(classificar_recomendacao, axis=1)



In [ ]:
df_toolings_cleaned.columns

In [ ]:
  # --- PREPARAÇÃO DOS DADOS (NOVOS PASSOS) ---

  # NOVO: Criar um mapa de {item: subfamilia} para consultas rápidas. É muito mais eficiente que usar .loc num loop.
  # Assumindo que as colunas são 'item' e 'SubFamilia'
  subfamilia_map = pd.Series(df_toolings_cleaned['SubFamily'].values, index=df_toolings_cleaned['Tool']).to_dict()

In [ ]:
import pandas as pd

def recomendar_itens_filtrado_subfamilia(carrinho, rules, df_itens_grupo, subfamilia_map, top_n=10):
    """
    Recomenda itens com base em regras de associação, excluindo itens da mesma sub-família
    e aplicando um boost para itens do mesmo grupo.
    """
    recomendados = {}
    carrinho_set = set(carrinho)

    # Obter o conjunto de sub-famílias dos itens que já estão no carrinho.
    subfamilias_carrinho = set(
        subfamilia_map.get(item) for item in carrinho_set if subfamilia_map.get(item) is not None
    )

    # Pega os grupos dos itens que já estão no carrinho (para a lógica de boost)
    grupos_carrinho = df_itens_grupo[df_itens_grupo['item'].isin(carrinho_set)]['grupo'].unique()

    for _, row in rules.iterrows():
        antecedents = set(str(a) for a in row['antecedents'])
        consequents = set(str(c) for c in row['consequents'])

        # Checa se a regra é ativada por um item no carrinho
        if len(carrinho_set & antecedents) > 0:
            for item in consequents:
                item_str = str(item)

                # Adicionado: Pula a recomendação se o item já estiver no carrinho
                if item_str in carrinho_set:
                    continue

                # --- LÓGICA DE FILTRAGEM ---
                # Pega a sub-família do item recomendado a partir do mapa
                item_subfamilia = subfamilia_map.get(item_str, 'Sub-família não definida')

                # Se a sub-família do item recomendado estiver na lista de sub-famílias do carrinho, PULA este item.
                if item_subfamilia in subfamilias_carrinho:
                    continue  # Vai para o próximo item no loop

                # --- LÓGICA DE SCORE (SÓ EXECUTA SE O ITEM NÃO FOR FILTRADO) ---
                score = row['lift']

                # Aplica boost se o item recomendado for do mesmo grupo de um item do carrinho
                grupo_item = df_itens_grupo.loc[df_itens_grupo['item'] == item_str, 'grupo']
                if not grupo_item.empty and grupo_item.values[0] in grupos_carrinho:
                    score = row['lift'] * 1.5  # Fator de boost

                # Adiciona ou atualiza a recomendação se o novo score for maior
                if item_str not in recomendados or score > recomendados[item_str]['score']:
                    recomendados[item_str] = {
                        'score': score,
                        'lift': row['lift'],
                        'classe': row.get('classe', None),
                        'subfamilia': item_subfamilia  # COMPLETADO: Armazena a sub-família que já tínhamos encontrado
                    }

    # Se não houver recomendações, retorna um DataFrame vazio com as colunas corretas
    if not recomendados:
        return pd.DataFrame(columns=['item', 'score', 'lift', 'classe', 'subfamilia'])

    # Converte o dicionário para DataFrame
    recomendados_df = pd.DataFrame([
        {
            'item': k,
            'score': v['score'],
            'lift': v['lift'],
            'classe': v['classe'],
            'subfamilia': v['subfamilia']  # COMPLETADO: Adiciona a coluna subfamilia ao DataFrame
        }
        for k, v in recomendados.items()
    ])

    # Ordena pelo score e retorna o top N
    recomendados_df = recomendados_df.sort_values('score', ascending=False).head(top_n).reset_index(drop=True)

    return recomendados_df

In [ ]:
def recomendar_itens_otimizado_grupo(carrinho, rules, df_itens_grupo, top_n=10):
    recomendados = {}
    carrinho_set = set(carrinho)

    # pega grupos do carrinho
    grupos_carrinho = df_itens_grupo[df_itens_grupo['item'].isin(carrinho_set)]['grupo'].unique()

    for _, row in rules.iterrows():
        antecedents = set([str(a) for a in row['antecedents']])
        consequents = set([str(c) for c in row['consequents']])

        # checa se regra aplica
        if len(carrinho_set & antecedents) > 0:
            for item in consequents:
                item_str = str(item)

                fator_boost = 1.0
                grupo_item = df_itens_grupo.loc[df_itens_grupo['item'] == item_str, 'grupo']

                # score inicial SEM boost
                score = row['lift']

                # aplica boost se for do mesmo grupo
                if len(grupo_item) > 0 and grupo_item.values[0] in grupos_carrinho:
                    fator_boost = 1.5
                    score = row['lift'] * fator_boost

                if item_str not in recomendados or score > recomendados[item_str]['score']:
                    recomendados[item_str] = {
                        'score': score,
                        'lift': row['lift'],
                        'classe': row.get('classe', None)  # previne erro caso 'classe' não exista
                    }

    # converte para DataFrame
    recomendados_df = pd.DataFrame([
        {'item': k, 'score': v['score'], 'lift': v['lift'], 'classe': v['classe']}
        for k, v in recomendados.items()
    ])

    # ordena pelo score e pega top N
    recomendados_df = recomendados_df.sort_values('score', ascending=False).head(top_n).reset_index(drop=True)

    return recomendados_df


In [ ]:
# ------------------------------
# 7. Teste de recomendação
# ------------------------------
carrinho = ['Alicate de Soldar completo']
recomendacoes = recomendar_itens_otimizado_grupo(carrinho, rules, df_itens_grupo, top_n=20)
print(recomendacoes)

In [ ]:
# ------------------------------
# 7. Teste de recomendação
# ------------------------------
carrinho = ['Alicate de Soldar completo']
recomendacoes = recomendar_itens_filtrado_subfamilia(carrinho, rules, df_itens_grupo,subfamilia_map, top_n=20)
recomendacoes

In [ ]:
carrinho = ['Aparelho de soldar TIG']
recomendacoes = recomendar_itens_otimizado_grupo(carrinho, rules, df_itens_grupo, top_n=10)
recomendacoes

In [ ]:
carrinho = ['Aparelho de soldar TIG']
recomendacoes = recomendar_itens_filtrado_subfamilia(carrinho, rules, df_itens_grupo, subfamilia_map, top_n=10)
recomendacoes

In [ ]:
carrinho = ['Rebolo (Retificadora direta)','Aparelho de soldar TIG']
recomendacoes = recomendar_itens_otimizado_grupo(carrinho, rules, df_itens_grupo, top_n=15)
print(recomendacoes)

In [ ]:
carrinho = ['Rebolo (Retificadora direta)','Aparelho de soldar TIG']
recomendacoes = recomendar_itens_filtrado_subfamilia(carrinho, rules, df_itens_grupo, subfamilia_map, top_n=10)
recomendacoes

In [ ]:
a

---

No teste acima, fizemos recomendações baseadas em regras de associação, aplicando as seguintes regras e estratégias:

1. **Identificação de conjuntos frequentes (FP-Growth)**

   * Usamos o FP-Growth para identificar itens que costumam ir juntos nos movimentos.
   * Consideramos apenas combinações de itens que aparecem em **pelo menos 1% dos movimentos** (`min_support=0.01`).

2. **Geração de regras de associação**

   * Criamos regras do tipo **A → B** usando a função `association_rules`.
   * **Lift** foi usado como métrica de seleção, que mede **quanto a presença do antecedente aumenta a chance do consequente aparecer**.
   * `min_threshold=2` → mantemos apenas regras com **lift maior que 2**, garantindo que as associações sejam relevantes.

3. **Classificação das regras por força**

   * **Compatibilidade forte**: lift > 3 e confiança > 70%
   * **Contextual**: lift entre 1 e 3
   * **Incompatíveis**: lift < 1

4. **Criação de grafo e detecção de comunidades**

   * Construímos um grafo onde os **nós são itens** e as **arestas representam regras fortes** (peso = lift).
   * Aplicamos **greedy_modularity_communities** para identificar grupos de itens que aparecem juntos com frequência.
   * Cada item foi associado a um grupo (`df_itens_grupo`).

5. **Otimização das recomendações com base em grupos**

   * Ao recomendar itens para um carrinho, verificamos se o **item candidato está no mesmo grupo** que os itens do carrinho.
   * Se estiver, aplicamos um **boost de 1.5x no score**, tornando a recomendação mais forte.
   * O score base é o **lift da regra**, podendo ser ajustado pelo boost.

6. **Classificação final das recomendações**

   * Cada recomendação recebe uma **classe** (`Compatibilidade forte`, `Contextual` ou `Incompatibilidade`) baseada nas métricas da regra.

7. **Retorno das recomendações**

   * O sistema retorna um **DataFrame com os itens recomendados**, ordenados pelo **score final**, limitando a quantidade (`top_n`) para sugestões mais relevantes.

---



### 1.1.5. 4ª EStratégia: Usando Word2Vec

In [ ]:
import re

def clean_tool_name(name):
    """
    Função para limpar e padronizar o nome de uma ferramenta.
    """
    # Converter para minúsculas e remover espaços nas pontas
    name = name.strip().lower()

    # Remover informações de perda, localização, datas, etc.
    # Ex: remove padrões como "-_perdido...", "-_sanga", "14/04/2023"
    name = re.sub(r'[-_]\s*perdido.*', '', name)
    name = re.sub(r'[-_]\s*sanga.*', '', name)
    name = re.sub(r'\d{2}/\d{2}/\d{4}', '', name)
    name = re.sub(r'[-_]\s*ct\s*\d+.*', '', name)
    name = re.sub(r'[-_]\s*danificado.*', '', name)
    name = re.sub(r'[-_]\s*obra.*', '', name)

    # Remover hifens ou underscores no final da string
    name = name.strip('-_ ')

    # Padronizar espaçamento para underscore
    name = re.sub(r'\s+', '_', name)

    # Exemplo de correção de erro comum (pode adicionar mais)
    if name.endswith('_m'):
        name = name + 'm' # Corrige '125_m' para '125_mm'

    return name

# Aplicar a limpeza na sua coluna 'Tool' ANTES de agrupar
df_toolings_isolate_cleaned['Tool_Cleaned'] = df_toolings_isolate_cleaned['Tool'].apply(clean_tool_name)

# AGORA, crie as suas transações (movimentos) com os nomes limpos
df_toolings_mov = df_toolings_isolate_cleaned.groupby('MOVEMENTID')['Tool_Cleaned'].apply(list).tolist()

# O resto do seu código de treino do Word2Vec continua igual,
# mas agora a usar esta lista `df_toolings_mov` limpa.

In [ ]:
df_toolings_mov

In [ ]:
!pip install gensim

In [ ]:
df_toolings_mov

In [ ]:
from gensim.models import Word2Vec

# Treinar embeddings
model = Word2Vec(
    sentences=df_toolings_mov,  # nossas “frases”
    vector_size=50,          # tamanho do embedding, ajustável
    window=5,                # janela de contexto (quantos itens à esquerda/direita)
    min_count=5,             # ignora itens raros
    workers=4,               # paralelismo
    sg=1                     # 1 = Skip-gram, bom para poucos dados; 0 = CBOW
)

# Verificar embedding de um item
item_vector = model.wv['retificadora_diam._125_mm']
print(item_vector.shape)  # (50,)


##### 1.1.5.1: Filtrar items de sub-famílias diferentes:

In [ ]:
df_mestra = df_toolings_isolate_cleaned[['Tool_Cleaned', 'SubFamily']]

# Crie o dicionário de mapeamento para consultas rápidas: {nome: subfamilia}
subfamilia_map = pd.Series(df_mestra['SubFamily'].values, index=df_mestra['Tool_Cleaned']).to_dict()

def recomendar_complementares_w2v(tool_name, model, subfamilia_map, top_n=10):
    """
    Gera recomendações do Word2Vec, excluindo itens da mesma sub-família.

    Args:
        tool_name (str): O nome canónico da ferramenta.
        model: O modelo Word2Vec treinado.
        subfamilia_map (dict): O dicionário que mapeia ferramentas para sub-famílias.
        top_n (int): O número final de recomendações desejado.

    Returns:
        list: Uma lista de tuplos (ferramenta, similaridade) filtrada.
    """
    # Obter a sub-família da ferramenta de entrada
    familia_input = subfamilia_map.get(tool_name)

    if not familia_input:
        print(f"Aviso: Ferramenta '{tool_name}' não encontrada no mapa de sub-famílias.")
        # Retorna as recomendações brutas se não soubermos a família
        try:
            return model.wv.most_similar(tool_name, topn=top_n)
        except KeyError:
            return []

    # Obter uma lista maior de recomendações brutas, pois algumas serão filtradas
    try:
        raw_recs = model.wv.most_similar(tool_name, topn=top_n * 3)
    except KeyError:
        print(f"Aviso: Ferramenta '{tool_name}' não está no vocabulário do modelo.")
        return []

    # Filtrar a lista
    filtered_recs = []
    for rec_tool, similarity in raw_recs:
        rec_familia = subfamilia_map.get(rec_tool)

        # A Regra: se a família da recomendação for diferente da família de entrada, mantenha.
        if rec_familia and rec_familia != familia_input:
            filtered_recs.append((rec_tool, similarity))

        # Parar quando atingirmos o número desejado
        if len(filtered_recs) == top_n:
            break

    return filtered_recs


In [ ]:
ferramenta_alvo = 'aparelho_de_soldar_tig'

# 1. Ver as recomendações BRUTAS (o que o Word2Vec puro faz)
print(f"--- Recomendações BRUTAS para '{ferramenta_alvo}' ---")
try:
    recs_brutas = model.wv.most_similar(ferramenta_alvo, topn=5)
    for item, score in recs_brutas:
        subfam = subfamilia_map.get(item, "N/A")
        print(f"- {item:<30} (Sub-família: {subfam})")
except KeyError:
    print(f"'{ferramenta_alvo}' não encontrado no modelo.")

print("\n" + "="*50 + "\n")

# 2. Ver as recomendações FILTRADAS (usando a nossa nova função)
print(f"--- Recomendações FILTRADAS para '{ferramenta_alvo}' (Itens Complementares) ---")
recs_filtradas = recomendar_complementares_w2v(ferramenta_alvo, model, subfamilia_map, top_n=5)
for item, score in recs_filtradas:
    subfam = subfamilia_map.get(item, "N/A")
    print(f"- {item:<30} (Sub-família: {subfam})")


In [ ]:
ferramenta_alvo = 'aparelho_de_soldar'

# 1. Ver as recomendações BRUTAS (o que o Word2Vec puro faz)
print(f"--- Recomendações BRUTAS para '{ferramenta_alvo}' ---")
try:
    recs_brutas = model.wv.most_similar(ferramenta_alvo, topn=5)
    for item, score in recs_brutas:
        subfam = subfamilia_map.get(item, "N/A")
        print(f"- {item:<30} (Sub-família: {subfam})")
except KeyError:
    print(f"'{ferramenta_alvo}' não encontrado no modelo.")

print("\n" + "="*50 + "\n")

# 2. Ver as recomendações FILTRADAS (usando a nossa nova função)
print(f"--- Recomendações FILTRADAS para '{ferramenta_alvo}' (Itens Complementares) ---")
recs_filtradas = recomendar_complementares_w2v(ferramenta_alvo, model, subfamilia_map, top_n=5)
for item, score in recs_filtradas:
    subfam = subfamilia_map.get(item, "N/A")
    print(f"- {item:<30} (Sub-família: {subfam})")


In [ ]:
import pandas as pd
# Supondo que o seu modelo Word2Vec já está treinado e carregado na variável 'model'
# Supondo que a sua tabela mestra está carregada em 'df_mestra'

# --- Preparação: Crie um dicionário para pesquisas rápidas ---
# É mais eficiente criar este mapa uma vez do que procurar no DataFrame a cada chamada
# O mapa terá o formato: {'nome_da_ferramenta': 'subfamilia'}
subfamilia_map = pd.Series(df_mestra['SubFamilia'].values, index=df_mestra['Nome_Canonico']).to_dict()


def recomendar_sem_mesma_subfamilia(tool_name, model, top_n=10):
    """
    Gera recomendações para uma ferramenta, excluindo itens da mesma sub-família.

    Args:
        tool_name (str): O nome canónico da ferramenta.
        model: O modelo Word2Vec treinado.
        top_n (int): O número final de recomendações desejado.

    Returns:
        list: Uma lista de tuplos (ferramenta, similaridade) filtrada.
    """
    # Passo 1: Obter a sub-família da ferramenta de entrada
    subfamilia_input = subfamilia_map.get(tool_name)

    if not subfamilia_input:
        print(f"Aviso: Ferramenta '{tool_name}' não encontrada na Tabela Mestra. Retornando recomendações brutas.")
        return model.wv.most_similar(tool_name, topn=top_n)

    # Passo 2: Obter uma lista maior de recomendações brutas
    # Pedimos mais (ex: 3x mais) porque sabemos que vamos descartar algumas.
    try:
        raw_recs = model.wv.most_similar(tool_name, topn=top_n * 3)
    except KeyError:
        return [] # Retorna lista vazia se a ferramenta não estiver no vocabulário do modelo

    # Passo 3 e 4: Filtrar a lista
    filtered_recs = []
    for rec_tool, similarity in raw_recs:
        # Obter a sub-família da ferramenta recomendada
        rec_subfamilia = subfamilia_map.get(rec_tool)

        # A regra de negócio: se a sub-família for diferente (e existir), adicionamos à lista
        if rec_subfamilia and rec_subfamilia != subfamilia_input:
            filtered_recs.append((rec_tool, similarity))

        # Parar quando tivermos o número desejado de recomendações
        if len(filtered_recs) == top_n:
            break

    return filtered_recs

# --- Exemplo de Uso ---

# Ferramenta de entrada
ferramenta_alvo = 'retificadora_125mm'

# Resultado ANTES do filtro (o que o Word2Vec puro retorna)
print(f"--- Recomendações BRUTAS para '{ferramenta_alvo}' ---")
recs_brutas = model.wv.most_similar(ferramenta_alvo, topn=5)
for item, score in recs_brutas:
    subfam = subfamilia_map.get(item, "N/A")
    print(f"- {item:<30} (Sub-família: {subfam})")

print("\n" + "="*50 + "\n")

# Resultado DEPOIS do filtro (usando a nossa nova função)
print(f"--- Recomendações FILTRADAS para '{ferramenta_alvo}' (excluindo mesma sub-família) ---")
recs_filtradas = recomendar_sem_mesma_subfamilia(ferramenta_alvo, model, top_n=5)
for item, score in recs_filtradas:
    subfam = subfamilia_map.get(item, "N/A")
    print(f"- {item:<30} (Sub-família: {subfam})")

## 1.2: Dados: `Agrupando por movimentos e desconsiderando ferramentas perdidas`

In [ ]:
df = pd.read_csv(r"/content/listagem_ferramentas_movimentacao_sem_container_sem_perdidas.csv")

In [ ]:
total_registos = len(df)
nomes_unicos = df['Tool'].nunique()

print(f"Total de registos de ferramentas: {total_registos}")
print(f"Número de nomes únicos: {nomes_unicos}")

- Remoção de ferramentas com nome incorreto:

In [ ]:
#remove columns with instruction to Don't use.
ids_to_drop = [5368, 27407, 29248, 5379, 5367, 5380, 7861, 19106]

df_cleaned = df[~df['ToolID'].isin(ids_to_drop)]

In [ ]:
total_registos = len(df_cleaned)
nomes_unicos = df_cleaned['Tool'].nunique()

print(f"Total de registos de ferramentas: {total_registos}")
print(f"Número de nomes únicos: {nomes_unicos}")

In [ ]:
# Aplique esta pré-limpeza
df_cleaned['Tool_Pre_Cleaned'] = df_cleaned['Tool'].apply(pre_clean_tool_name)

# Veja a magia acontecer:
novos_nomes_unicos = df_cleaned['Tool_Pre_Cleaned'].nunique()
print(f"Número de nomes únicos ANTES da pré-limpeza: {nomes_unicos}")
print(f"Número de nomes únicos DEPOIS da pré-limpeza: {novos_nomes_unicos}")

In [ ]:
# --- ETAPA 1: CRIAR O FICHEIRO DE ENTRADA ---

# Contar a frequência dos nomes pré-limpos que você acabou de criar
pre_cleaned_counts = df_cleaned['Tool_Pre_Cleaned'].value_counts()

# O nome do ficheiro que o próximo script vai usar como entrada
NOME_FICHEIRO_INPUT = 'nomes_para_mapear_sem_perdidas.csv'

# Exportar para um ficheiro CSV. Este ficheiro terá duas colunas: o nome e a contagem.
pre_cleaned_counts.to_csv(NOME_FICHEIRO_INPUT)

print(f"\nFicheiro de entrada '{NOME_FICHEIRO_INPUT}' criado com sucesso!")
print("Agora, vamos usar este ficheiro na Etapa 2.")

In [ ]:
# Conteúdo do ficheiro: agrupar_ferramentas.py

import pandas as pd
from fuzzywuzzy import fuzz

# --- CONFIGURAÇÕES ---
# O nome do ficheiro que você acabou de criar na Etapa 1
INPUT_CSV = 'nomes_para_mapear_sem_perdidas.csv'
# O nome do ficheiro de saída que este script vai gerar
OUTPUT_CSV = 'mapa_ferramentas_sugerido_sem_perdidas.csv'
# Limiar de similaridade. 85 é um bom valor inicial.
SIMILARITY_THRESHOLD = 85

# --- LÓGICA DO SCRIPT ---

print("Iniciando o script de agrupamento...")

# 1. Carregar os nomes únicos pré-limpos
try:
    df_counts = pd.read_csv(INPUT_CSV)
    df_counts.columns = ['Nome_Pre_Limpado', 'Contagem']
except FileNotFoundError:
    print(f"ERRO: O ficheiro '{INPUT_CSV}' não foi encontrado. Verifique se ele existe na pasta.")
    exit()

df_counts = df_counts.sort_values(by='Contagem', ascending=False)
tools_to_process = df_counts['Nome_Pre_Limpado'].tolist()

print(f"Agrupando {len(tools_to_process)} nomes únicos...")

mapping_list = []

while tools_to_process:
    anchor_name = tools_to_process.pop(0)

    mapping_list.append({
        'Nome_Sujo': anchor_name,
        'Nome_Canonico_Sugerido': anchor_name,
        'Similaridade': 100
    })

    matches = [
        tool for tool in tools_to_process
        if fuzz.token_set_ratio(anchor_name, tool) > SIMILARITY_THRESHOLD
    ]

    for match in matches:
        similarity_score = fuzz.token_set_ratio(anchor_name, match)
        mapping_list.append({
            'Nome_Sujo': match,
            'Nome_Canonico_Sugerido': anchor_name,
            'Similaridade': similarity_score
        })
        tools_to_process.remove(match)

    if len(tools_to_process) % 200 == 0 and len(tools_to_process) > 0:
        print(f"Faltam {len(tools_to_process)} nomes para processar...")

df_map_sugestao = pd.DataFrame(mapping_list)
df_map_sugestao.to_csv(OUTPUT_CSV, index=False)

print(f"\nCONCLUÍDO! O mapa de sugestões foi salvo em '{OUTPUT_CSV}'.")
print("O seu próximo passo é abrir este ficheiro no Excel/Sheets para rever e corrigir.")

## 1.3 Dados: `Agrupando por Obra e não por movimento`

In [ ]:
# read data
tooling_jobid = pd.read_csv(r"/content/toolings_per_jobid.csv")

In [ ]:
tooling_jobid

In [ ]:
tooling_original = tooling_jobid.copy()

In [ ]:
tooling_jobid = tooling_jobid.groupby('DESTINATIONJOBID').agg(
    {
        'Tool': lambda x: list(x),
        'TotalVolume': 'sum'
    }
).reset_index()


In [ ]:
tooling_jobid

In [ ]:
# filtrar apenas obras com mais de uma ferramenta associada

tooling_jobid = tooling_jobid[tooling_jobid['Tool'].apply(len) > 1]

In [ ]:
tooling_jobid.shape

In [ ]:
from collections import Counter

# 1. Juntar todas as ferramentas numa única lista
# O método 'explode' é perfeito para transformar listas em linhas separadas
all_tools = tooling_jobid['Tool'].explode().tolist()

# 2. Contar a frequência de cada ferramenta
tool_counts = Counter(all_tools)

# 3. Ver as ferramentas mais comuns
# O método 'most_common()' retorna uma lista de tuplos (ferramenta, contagem)
print("As 10 ferramentas mais frequentes:")
for tool, count in tool_counts.most_common(20):
    print(f"- {tool}: {count} vezes")

In [ ]:
# Transformar a lista de ferramentas em linhas individuais
df_exploded = tooling_jobid.explode('Tool')

# Criar a matriz one-hot encoded
basket_sets = pd.crosstab(df_exploded['DESTINATIONJOBID'], df_exploded['Tool'])

# Converter os valores para 0 ou 1
def encode_units(x):
    if x >= 1:
        return 1
    return 0

basket_sets = basket_sets.apply(lambda col: col.map(encode_units))

print("Formato One-Hot Encoded (amostra):")
print(basket_sets.head())

In [ ]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import pandas as pd

# ------------------------------
# 1. Frequent itemsets
# ------------------------------
frequent_itemsets = fpgrowth(basket_sets, min_support=0.1, use_colnames=True)
print(frequent_itemsets.head())


In [ ]:

# ------------------------------
# 2. Association rules
# ------------------------------
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)

# ordenar pelas regras mais interessantes
rules = rules.sort_values(["lift", "confidence"], ascending=False).reset_index(drop=True)

print("Regras geradas:", rules.shape[0])
print(rules.head(10))

# ------------------------------
# 3. Classificação por lift/confiança
# ------------------------------
strong_rules = rules[(rules['lift'] > 3) & (rules['confidence'] > 0.7)].copy()
contextual_rules = rules[(rules['lift'] > 1) & (rules['lift'] <= 3)].copy()
incompatibles = rules[rules['lift'] < 1].copy()



In [ ]:
# ------------------------------
# 4. Criar grafo e detectar comunidades
# ------------------------------
G = nx.Graph()
for _, row in strong_rules.iterrows():
    # antecedents e consequents são frozenset, converte para string
    antecedents = [str(a) for a in row['antecedents']]
    consequents = [str(c) for c in row['consequents']]
    for ant in antecedents:
        for cons in consequents:
            G.add_edge(ant, cons, weight=row['lift'])

communities = list(greedy_modularity_communities(G))

# ------------------------------
# 3. Criar DataFrame de itens com seus grupos
# ------------------------------
item_para_grupo = {}
for i, comm in enumerate(communities):
    for item in comm:
        item_para_grupo[item] = i

df_itens_grupo = pd.DataFrame({
    'item': list(item_para_grupo.keys()),
    'grupo': list(item_para_grupo.values())
})

